# Control de versiones con git

**Curso**: Ciencia de Datos en Python

**Catedrático**: Ing. Luis Leal

**Estudiante**: Dany Rafael Díaz Lux (21000864)

**Enunciado**: Tomar la [lección del MIT de git](https://missing.csail.mit.edu/2020/version-control/) y en un jupyter notebook entregar un ensayo acerca de git, es decir: un resumen de lo que ha aprendido, conclusiones, comentarios y ejemplos propios, esto usando markdown en el notebook , para ejemplificar cualquier comando de git pueden usar [magic cells](https://jakevdp.github.io/PythonDataScienceHandbook/01.03-magic-commands.html), un tipo especial de celdas de jupyter que nos permite escribir comandos que no son de python si no de nuestro sistema.

**Fecha de entrega**: miércoles 10 de febrero.

## Resumen

### Introducción
Los sistemas de control de versiones (VCSs por sus siglas en inglés) ayudan a llevar seguimiento de los cambios que se aplican a un conjunto de archivos. Ayudan mucho en proyectos donde colaboran muchas personas en diferentes archivos, o incluso en cuando estás trabajando en solitario porque VCSs ayudan a contestar preguntas importantes como:
* ¿__Quién__ cambió este archivo?
* ¿__Cuándo__ fue introducido este cambio?
* ¿__Por qué__ fue cambiada esta línea?
* _Muchas más_...

### Entendiendo git
git es un VCS muy popular que regularmente se empieza a usar sin entender por completo cómo funciona, esto puede conllevar a utilizar sólo los comandos más básicos de git sin mayor comprensión de lo que pasa con nuestros archivos y nos impiden resolver problemas más complejos que pueden presentarse más adelante.

Para mejorar nuestra compresión de git iniciaremos con una explicación de su modelo de datos.

### Modelo de datos de git

#### _Snapshots_ (instantáneas)
git modela la historia de un conjunto de archivos, dentro de un folder principal, como **una serie de _snapshots_**. Para entender un _snapshot_, se deben comprender los siguientes términos:
* _blob_: un archivo, un conjunto de bytes.
* _tree_ (árbol): un folder, que puede contener _blobs_ o _trees_.

Un _snapshot_ es el folder principal (que contiene todos los _blobs_ y _trees_) que está siendo registrado por git.

### Modelando la historia = relacionando _snapshots_
Para manejar la historia de todos los archivos, git usa un _Grafo dirigido acíclico_ (DAG por sus siglas en inglés) de _snapshots_, que básicamente significa que cada _snapshot_ en git hará referencia a uno o varios _snapshots_ padres. Como lo muestra la siguiente imagen:

![Grafo dirigido acíclico de git](https://www.oreilly.com/library/view/git-pocket-guide/9781449327507/images/gtpg_0101.png "Grafo dirigido acíclico de git")

_Imagen obtenida de la página web: [Chapter 1. Understanding Git](https://www.oreilly.com/library/view/git-pocket-guide/9781449327507/ch01.html), como se puede observar, cada nodo en el grafo (que representa un snapshot) apunta al menos a un padre, con la excepción del nodo inicial que representa el inicio de la historia de los archivos en git._

Cada _snapshot_ se realiza por medio de un _commit_ (comprometer) en git, que guarda los contenidos de todos los archivos en ese momento específico.

### Objetos y direccionamiento de contenido
Un objeto puede ser un _blob_ (archivo), _tree_ (folder), o un _commit_ (instantánea). En el almacén de datos de git todo el contenido de los  objetos pueden ser obtenidos por un identificador generado por la aplicación del algoritmo [SHA1](https://en.wikipedia.org/wiki/SHA-1) en dicho objeto.

Es importante señalar que cuando un objeto hace referencia a otro objeto, **no posee el contenido del objeto referenciado, sólo posee el identificador a ese objeto**. Así por ejemplo, un _tree_ (folder) sólo posee los identificadores a los objetos que contiene.

### Referencias
Los identificadores generados por el algoritmo **SHA1** están formados por 40 caracteres alfanuméricos (como `a43913adf3occmce...`), lo cual no es conveniente para las personas. Por lo cual git permite utilizar **referencias** que son nombres que podemos darle a cualquier _commit_, como por ejemplo `Version2021`. Estas referencias pueden ser actualizadas para apuntar a nuevos o anteriores _commits_.

Existe una referencia especial **que indica donde nos encontramos siempre** en nuestro grafo de git, esta referencia se llama **HEAD**.

### Repositorios
Un repositorio de git son los objetos de datos y referencias. En eso consiste básicamente el modelo de datos de git. Todos los comandos de git hacen manipulaciones al _DAG_ por medio de agregar objetos y agregar o actualizar referencias.

### _Staging area_ (área de preparación)
_Staging area_ es aquella en la cual se incluyen todos los cambios que deseamos incluir en un _commit_. El objetivo de _staging area_ es realizar commit con el estado de los archivos que deseamos. Por ejemplo, si tenemos 2 archivos modificados, pero sólo deseamos incluir las modificaciones de un archivo en el próximo _commit_, incluimos sólo este archivo en nuestra _staging area_. El otro archivo no incluido se puede dejar para más adelante o descartar de acuerdo a nuestras necesidades.

## Ejemplos
A continuación se mostrarán diversos comandos de git para indicar su funcionamiento y utilidad.

**_Nota: 
Todos los comandos a continuación fueron ejecutados en un sistema Windows 10, con diversas utilidades adicionales instaladas (como git bash), por lo que si un comando no funciona adecuadamente podría deberse a problemas específicos de sistema operativo o utilidades que no se tienen instaladas. Además los comandos se deben ejecutar en el orden que se muestran a continuación, pues algunos de ellos dependen de comandos ejecutados anteriormente._**

### 1. git init
Crea un nuevo repositorio de git

In [7]:
%sx git init _nuevo_repositorio_test_

['Initialized empty Git repository in C:/Users/Dany/Documents/01 Dany/MaestrÃ\xada/Python para ciencia de datos/python-ciencia-de-datos/Tarea 2 - git y control de versiones/_nuevo_repositorio_test_/.git/']

### 2. git status
Señala el estado de nuestro repositorio actualmente. Qué archivos han cambiado. Qué archivos están dentro y fuera de _staging area_, etc.

In [18]:
# Enter to the new repository created
%cd _nuevo_repositorio_test_

# Create a new file in the repository initialized
%sx touch archivo_de_prueba.txt

# Add some content to the file
%sx echo Listado de cosas: > archivo_de_prueba.txt
%sx echo Sol >> archivo_de_prueba.txt

# Ejecutar git status para ver el estado actual del repositorio
%sx git status

C:\Users\Dany\Documents\01 Dany\Maestría\Python para ciencia de datos\python-ciencia-de-datos\Tarea 2 - git y control de versiones\_nuevo_repositorio_test_


['On branch master',
 '',
 'No commits yet',
 '',
 'Untracked files:',
 '  (use "git add <file>..." to include in what will be committed)',
 '',
 '\tarchivo_de_prueba.txt',
 '',
 'nothing added to commit but untracked files present (use "git add" to track)']

### 3. git add
Incluye archivos al _staging area_ (en este espacio estarán todos los archivos que serán tomados en cuenta al momento de hacer un _commit_).

In [19]:
# Incluimos archivo_de_prueba.txt al staging area
%sx git add archivo_de_prueba.txt

# Consultamos de nuevo el estado del repositorio
%sx git status

['On branch master',
 '',
 'No commits yet',
 '',
 'Changes to be committed:',
 '  (use "git rm --cached <file>..." to unstage)',
 '',
 '\tnew file:   archivo_de_prueba.txt',
 '']

### 4. git commit
Crea una instantánea de todos los archivos que conforman el repositorio.

In [20]:
%sx git commit -m "Nuevo archivo en repositorio."

['[master (root-commit) 8ef5d04] Nuevo archivo en repositorio.',
 ' 1 file changed, 2 insertions(+)',
 ' create mode 100644 archivo_de_prueba.txt']

### 5. git log
Muestra todos los _commits_ realizados en el historial: quién lo hizo, cuándo se hizo, y un mensaje asociado con el commit.

In [21]:
%sx git log

['commit 8ef5d0484c5b25d86aeb299e1a9a5963c0796f64',
 'Author: Dany DÃ\xadaz <dany_diaz_22@hotmail.com>',
 'Date:   Sun Feb 7 09:31:33 2021 -0600',
 '',
 '    Nuevo archivo en repositorio.']

### 6. git diff
Nos permite ver diferencias entre _commits_, entre nuestro directorio de trabajo actual y nuestra _staging area_, entre _staging area_ y último _commit_, etc; para todos los archivos o un archivo en particular.

* **git diff**: Sin parámetros nos permite observar las diferencias que existen en nuestro directorio actual que no existen en nuestro _staging area_.
* **git diff --staged**: Muestra las diferencias entre nuestro _staging area_ y nuestro último _commit_.
* **git diff [commit1] [commit2]**: Muestra las diferencias entre _commit1_ y _commit2_.

In [22]:
# Agregar una línea a archivo_de_prueba.txt
%sx echo Campo >> archivo_de_prueba.txt

#Mostrar diferencias entre nuestro directorio actual y nuestro staging area.
%sx git diff

['diff --git a/archivo_de_prueba.txt b/archivo_de_prueba.txt',
 'index bd755bc..c48166e 100644',
 '--- a/archivo_de_prueba.txt',
 '+++ b/archivo_de_prueba.txt',
 '@@ -1,2 +1,3 @@',
 ' Listado de cosas: ',
 ' Sol ',
 '+Campo ']

In [23]:
# Agregar todos nuestros cambios al staging area.
%sx git add -A

# Mostrar los cambios entre nuestro staging area y nuestro último commit
%sx git diff --staged

['diff --git a/archivo_de_prueba.txt b/archivo_de_prueba.txt',
 'index bd755bc..c48166e 100644',
 '--- a/archivo_de_prueba.txt',
 '+++ b/archivo_de_prueba.txt',
 '@@ -1,2 +1,3 @@',
 ' Listado de cosas: ',
 ' Sol ',
 '+Campo ']

In [28]:
# Realizar commit de nuestro último cambio.
%sx git commit -m "Nueva línea ingresada."

# Comparar los cambios entre los últimos dos commits.
# Notar que HEAD~1 hace referencia al commit anterior al que ahora apunta HEAD.
%sx git diff HEAD~1 HEAD

['diff --git a/archivo_de_prueba.txt b/archivo_de_prueba.txt',
 'index bd755bc..c48166e 100644',
 '--- a/archivo_de_prueba.txt',
 '+++ b/archivo_de_prueba.txt',
 '@@ -1,2 +1,3 @@',
 ' Listado de cosas: ',
 ' Sol ',
 '+Campo ']

### 7. git branch
Muestra las _branches_ (ramas) que existen en nuestro repositorio. También permite crear una nueva branch en el _commit_ actual que nos encontramos.

In [29]:
# Mostrar las branches que existen. Por defecto existirá desde el inicio una branch "master".
%sx git branch

['* master']

In [30]:
# Crear una nueva branch desde el commit que nos encontramos
%sx git branch NuevaRama

# Mostrar nuevamente todas las ramas
%sx git branch

['  NuevaRama', '* master']

### 8. git checkout
Cambia nuestra posición (HEAD) entre _branches_ o a cualquier _commit_ en nuestro historial del repositorio. Al cambiar a dicha _branch_ o _commit_ todos los contenidos de los archivos en el repositorio son restablecidos para coincidir con ese _commit_.

In [31]:
# Cambiar HEAD a la nueva branch: NuevaRama
%sx git checkout NuevaRama

["Switched to branch 'NuevaRama'"]

In [33]:
# Mostrar ramas. Notar que el asterisco en NuevaRama, señala que ahora HEAD está apuntando a Nueva Rama
%sx git branch

['* NuevaRama', '  master']

In [37]:
# En la nueva branch, agregar nueva línea a archivo_de_prueba.txt
%sx echo Agua >> archivo_de_prueba.txt

# Agregar cambio a staging area
%sx git add -A

# Nuevo commit sobre branch NuevaRama
%sx git commit -m "Agregar agua a listado."

# Mostrar historial de repositorio
# Notar que HEAD está apuntando a "NuevaRama" y está un commit adelante de la branch "master"
%sx git log --graph --oneline --decorate

['* b3f3017 (HEAD -> NuevaRama) Agregar agua a listado.',
 '* bfb7aeb (master) Nueva lÃ\xadnea ingresada.',
 '* 8ef5d04 Nuevo archivo en repositorio.']

In [38]:
# Ver contenido de archivo_de_prueba.txt en branch "NuevaRama"
# Notar que existe la línea con contenido "Agua"
%sx cat archivo_de_prueba.txt

['Listado de cosas: ', 'Sol ', 'Campo ', 'Agua ']

In [39]:
# Cambiar a branch "master"
%sx git checkout master

# Al ver nuevamente el contenido de archivo_de_prueba.txt vemos que no existe la línea "Agua" porque dicha línea no existe en branch master
%sx cat archivo_de_prueba.txt

['Listado de cosas: ', 'Sol ', 'Campo ']

In [41]:
# Mostrar historial de repositorio de git
# Notar que HEAD está apuntando a master.
%sx git log --all --graph --oneline --decorate

['* b3f3017 (NuevaRama) Agregar agua a listado.',
 '* bfb7aeb (HEAD -> master) Nueva lÃ\xadnea ingresada.',
 '* 8ef5d04 Nuevo archivo en repositorio.']

### 9. git merge
Nos permite unir dos o más _branches_.

In [42]:
# Hacer merge de las ramas "master" (en la cual nos encontramos) y la branch "NuevaRama"
# Notar que git realizó un "fast-forward" que significa que "master" fue actualizada con los cambios de "NuevaRama"
# ya que no había ningún conflicto y toda la historia de "master" estaba contenida en "NuevaRama"
%sx git merge NuevaRama

['Updating bfb7aeb..b3f3017',
 'Fast-forward',
 ' archivo_de_prueba.txt | 1 +',
 ' 1 file changed, 1 insertion(+)']

In [43]:
# Ver historial de nuestro repositorio.
# Notar que ahora HEAD, master, NuevaRama, apuntan al mismo commit.
%sx git log --all --graph --oneline --decorate

['* b3f3017 (HEAD -> master, NuevaRama) Agregar agua a listado.',
 '* bfb7aeb Nueva lÃ\xadnea ingresada.',
 '* 8ef5d04 Nuevo archivo en repositorio.']

### 10. git rebase
También puede unir dos _branches_, pero a diferencia de merge, su objetivo es aplicar los _commits_ de una branch **después** de los _commits_ de otra branch desde un punto común en su historia.

![Diferencia entre git merge y git rebase](https://i.stack.imgur.com/fb6L4.png "Diferencia entre git merge y git rebase")

_Imagen obtenida de la página web: [When do you use Git rebase instead of Git merge? - closed](https://stackoverflow.com/questions/804115/when-do-you-use-git-rebase-instead-of-git-merge), se puede observar que aunque ambas operaciones dejarán un punto común final a ambas branches, git rebase mostrará todos los commits de "feature" después de los commits de "master" mientras que git merge los mostrará de forma paralela._

## Comentarios personales

## Conclusiones